##### Copyright 2024 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: JSON Mode Quickstart

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/JSON_mode.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

The Gemini API can be used to generate a JSON output if you set the schema that you would like to use.

**Note**: JSON schemas are only supported by Gemini 1.5 Pro right now.

In [ ]:
!pip install -U -q google-generativeai

In [2]:
import google.generativeai as genai

import json
import dataclasses
import typing_extensions as typing

## Configure your API key

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## Activate JSON Mode

 Activate JSON mode by specifying `respose_mime_type` in the `generation_config` parameter:

In [3]:
model = genai.GenerativeModel("gemini-1.5-flash-latest",
                              generation_config={"response_mime_type": "application/json"})

For this first example just describe the schema you want back:

In [4]:
prompt = """List a few popular cookie recipes using this JSON schema:

Recipe = {'recipe_name': str}
Return: list[Recipe]"""

In [5]:
raw_response = model.generate_content(prompt)

Parse the string to JSON:

In [6]:
response = json.loads(raw_response.text)
print(response)

[{'recipe_name': 'Chocolate Chip Cookies'}, {'recipe_name': 'Oatmeal Raisin Cookies'}, {'recipe_name': 'Snickerdoodles'}, {'recipe_name': 'Sugar Cookies'}, {'recipe_name': 'Peanut Butter Cookies'}]


For readability serialize and print it:

In [7]:
print(json.dumps(response, indent=4))

[
    {
        "recipe_name": "Chocolate Chip Cookies"
    },
    {
        "recipe_name": "Oatmeal Raisin Cookies"
    },
    {
        "recipe_name": "Snickerdoodles"
    },
    {
        "recipe_name": "Sugar Cookies"
    },
    {
        "recipe_name": "Peanut Butter Cookies"
    }
]


## Generate JSON from schema

While `gemini-1.5-flash` models only accept a text description of the JSON you want back, `gemini-1.5-pro` models support "controlled generation" (aka "constrained decoding"). This allows you to pass a schema object (or a python type equivalent) and the output will strictly follow that schema.

Following the same example as the previous section, here's that recipe type:

In [8]:
class Recipe(typing.TypedDict):
    recipe_name: str

For this example you want a list of `Recipe` objects, so pass `list[Recipe]` to the `response_schema` field of the `generation_config`.

In [9]:
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")

result = model.generate_content(
    "List a few popular cookie recipes",
    generation_config=genai.GenerationConfig(
        response_mime_type="application/json",
        response_schema = list[Recipe]),
    request_options={"timeout": 600},
)

In [10]:
print(result.text)

[{"recipe_name": "Chocolate Chip Cookies"}, {"recipe_name": "Peanut Butter Cookies"}, {"recipe_name": "Oatmeal Raisin Cookies"}, {"recipe_name": "Sugar Cookies"}, {"recipe_name": "Snickerdoodles"}] 
